In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data
)

company_name = "Lego"

In [2]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Lego...
Using existing context report for Lego.
Context data for Lego generated: {'company_name': 'Lego', 'count_employee': 550, 'count_department': 25, 'count_customer': 55, 'count_product': 180, 'count_procurement': 140, 'count_service': 140, 'count_account': 60, 'count_vendor': 75, 'estimated_product': 105000000, 'estimated_service': 40000000, 'estimated_overhead': 442000000, 'estimated_revenue': 649000000}
Generating dimensions for company: Lego.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Accounts data generated.
✔ Customers data generated.
✔ Departments data generated.
✔ Vendors data generated.
✔ Payroll data generated.
✔ All table lengths validated.
✔ All CSVs saved to: data/outputdata

 * Semantic mapping started * :
Time estimate: 3-5 minutes
✔ Procurement mapping done!
✔ Service mapping done!
✔ Product mapping done!
✔ All mapping 

In [3]:
now = dt.now()
version_tag = (
    company_name.lower()
    + "_date_" + now.strftime("%m%d")
    + "_time_" + now.strftime("%H%M%S")
)
execute_db_operations(version_tag)

[INFO] Inserting 25 rows into dim_department using to_sql...
[INFO] Inserting 55 rows into dim_customer using to_sql...
[INFO] Inserting 180 rows into dim_product using to_sql...
[INFO] Inserting 60 rows into dim_account using to_sql...
[INFO] Inserting 140 rows into dim_procurement using to_sql...
[INFO] Inserting 140 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_payline using to_sql...
[INFO] Inserting 75 rows into dim_vendor using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 550 rows into dim_employee using to_sql...
[INFO] Inserting 554400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain 

[INFO] Inserting 49920 rows into fact_general_ledger using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain 

[INFO] Inserting 33355 rows into fact_budget using to_sql...


In [12]:
def generate_revenue_items_llm(company_name: str, count: int = 100, model: str = "gpt-4.1", temp: float = 0.8):
    """
    Generate revenue items (e.g. product/service revenue categories) for a company.
    Each item will be mapped to a Revenue-type account in the Chart of Accounts.
    """

    client = prompt_utils.get_openai_client()
    over_request_count = int(np.floor(int(count) * 1.4))

    header = "name;revenue_segment;proportionality"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    prompt = f"""
    You are a financial analyst and ERP expert.
    Generate a realistic ranked list of {over_request_count} **revenue items** for {company_name}, Denmark-only.

    Fields:
    - name = revenue line item (e.g., "Running Shoes Sales", "Consulting Services", "Subscription Fees")
    - revenue_segment = category (e.g., Product Revenue, Service Revenue, Licensing, Subscription, Consulting)
    - proportionality = share of total company revenue

    Rules:
    - Cover both major product revenues and service/other revenues where applicable.
    - Use realistic naming aligned with company operations/industry.
    - Ensure categories can map logically to Chart of Accounts entries of type 'Revenue'.
    - Rank by proportionality (desc).
    - Avoid vague terms like "Miscellaneous Revenue".

    For context, here is a short version of the latest year-end report for {company_name}:
    {ctxb}

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful financial analyst and ERP mapping assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=temp,
    )

    df_revenue = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

    # Add unique IDs for the revenue items
    df_revenue.insert(0, "revenue_id", range(50, len(df_revenue) + 50))

    # Convert proportionalities to percentages (scaled to 1.0)
    df_revenue = utils.convert_column_to_percentage(df_revenue, "proportionality", scale=1.0)

    return df_revenue


In [13]:
def generate_cogs_items_llm(company_name: str, count: int = 100, model: str = "gpt-4.1", temp: float = 0.8):
    """
    Generate COGS (Cost of Goods Sold) items for a company.
    Each item should map to Product Expense or Service Expense accounts in the COA.
    """

    client = prompt_utils.get_openai_client()
    over_request_count = int(np.floor(int(count) * 1.4))

    header = "name;cogs_segment;proportionality"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    prompt = f"""
    You are a financial analyst and ERP expert.
    Generate a realistic ranked list of {over_request_count} **COGS (Cost of Goods Sold) items** 
    for {company_name}, Denmark-only.

    Fields:
    - name = specific COGS line item (e.g., "Raw Materials", "Packaging", "Shipping Costs", "Outsourced Manufacturing", "Implementation Services")
    - cogs_segment = category (e.g., Product COGS, Service Delivery Costs, Logistics, Manufacturing Support)
    - proportionality = share of total COGS

    Rules:
    - Ensure items logically map to Chart of Accounts entries of type "Product Expense" or "Service Expense".
    - Cover both direct material/product-related COGS and service/operational delivery costs.
    - Include raw materials, components, packaging, logistics, service subcontracting, hosting, etc.
    - Rank by proportionality (desc).
    - Avoid vague placeholders like "Miscellaneous Costs".

    For context, here is a short version of the latest year-end report for {company_name}:
    {ctxb}

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful financial analyst and ERP mapping assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=temp,
    )

    df_cogs = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

    # Add unique IDs for the COGS items
    df_cogs.insert(0, "cogs_id", range(200, len(df_cogs) + 200))

    # Normalize proportionalities
    df_cogs = utils.convert_column_to_percentage(df_cogs, "proportionality", scale=1.0)

    return df_cogs

def generate_ebit_items_llm(company_name: str, count: int = 100, model: str = "gpt-5", temp: float = 0.9):
    """
    Generate realistic EBIT (Operating Income & Expense) items for a company.
    These represent the main operating costs and incomes between Gross Profit and Operating Profit.
    """

    client = prompt_utils.get_openai_client()
    over_request_count = int(np.floor(int(count) * 1.4))

    header = "name;ebit_segment;proportionality"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    prompt = f"""
    {ctxb}

    You are a financial analyst and ERP data modeler.
    Generate a realistic ranked list of {over_request_count} **EBIT items (Operating Income & Expenses)** 
    for {company_name}, Denmark-only.

    Fields:
    - name = EBIT line item
    - ebit_segment = category (e.g., Selling & Distribution, Administration, Production Overhead, Other Operating Income)
    - proportionality = share of total EBIT impact (absolute value, 0–1 range)

    Rules:
    - Include both operating costs and operating income directly below gross profit.
    - Exclude financing, tax, and extraordinary items.
    - Reflect realistic operating structure for the given company and industry.
    - Rank by proportionality (descending).
    - Avoid placeholders such as "Miscellaneous expenses" or "Other income".

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful ERP and finance assistant who outputs clean CSV tables for P&L modeling."},
            {"role": "user", "content": prompt},
        ],
        temperature=temp,
    )

    df_ebit = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

    # Assign IDs
    df_ebit.insert(0, "ebit_id", range(400, 400 + len(df_ebit)))

    # Normalize proportionalities
    df_ebit = utils.convert_column_to_percentage(df_ebit, "proportionality", scale=1.0)

    return df_ebit


def generate_inventory_items_llm(company_name: str, count: int = 100, model: str = "gpt-5", temp: float = 0.9):
    """
    Generate realistic Inventory items for a company.
    Each item should correspond to balance sheet Inventory accounts such as Raw Materials, WIP, or Finished Goods.
    """

    client = prompt_utils.get_openai_client()
    over_request_count = int(np.floor(int(count) * 1.4))

    header = "name;inventory_segment;proportionality"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    prompt = f"""
    {ctxb}

    You are a financial analyst and ERP data modeler.
    Generate a realistic ranked list of {over_request_count} **Inventory items** for {company_name}, Denmark-only.

    Fields:
    - name = specific inventory line item
    - inventory_segment = category (e.g., Raw Materials, Work in Progress, Finished Goods, Spare Parts, Consumables)
    - proportionality = share of total inventory value (0–1 range)

    Rules:
    - Must map logically to balance sheet accounts of type "Inventory".
    - Include tangible goods (raw materials, semi-finished, finished goods, packaging, spare parts, etc.)
      and work-in-progress where relevant.
    - Reflect realistic items for the company's industry and geography.
    - Rank by proportionality (descending).
    - Avoid vague or generic terms such as "Other inventory" or "Miscellaneous stock".

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful ERP and finance domain assistant who outputs clean CSV tables for balance sheet data models."},
            {"role": "user", "content": prompt},
        ],
        temperature=temp,
    )

    df_inventory = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

    # Assign unique IDs
    df_inventory.insert(0, "inventory_id", range(300, 300 + len(df_inventory)))

    # Normalize proportionalities to sum to 1.0
    df_inventory = utils.convert_column_to_percentage(df_inventory, "proportionality", scale=1.0)

    return df_inventory


def tailor_coa_to_lego(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    replacements = {
        "Raw Materials": "ABS Plastic & Pigments",
        "Finished Goods": "Finished LEGO Sets",
        "Work in Progress": "Moulding & Assembly in Progress",
        "Buildings & Improvements": "LEGO Factories & Warehouses",
        "Leasehold Improvements": "Retail Store Fit-outs",
        "Machinery & Equipment": "Moulding Machines & Production Lines",
        "Retail Gross Sales": "LEGO Retail Store Sales",
        "Trade Gross Sales": "Wholesale Channel Sales",
        "Retail COS": "Production Cost – Retail",
        "Trade COS": "Manufacturing Cost – Trade",
        "Marketing Collateral": "Packaging & Campaign Materials",
        "Other Income": "Royalty & Licensing Income",
    }
    df["AccountDescription"] = df["AccountDescription"].replace(replacements)
    df["GLLevel05"] = df["GLLevel05"].replace(replacements)
    return df


In [ ]:
MASTER_PROMPT = """
You are a financial controller generating a realistic analytical spend dataset for a company.

### INPUT DATA
Company: {company_name}

Financial Totals (DKK):
- Revenue: {revenue_value}
- COGS: {cogs_value}
- Gross Profit: {gross_profit_value}
- Fixed Cost: {fixed_cost_value}
- EBIT: {ebit_value}

Chart of Accounts (CoA):
{coa_list}

Row Counts:
- Revenue rows: {revenue_rows}
- COGS rows: {cogs_rows}
- FixedCost rows: {fixedcost_rows}
- EBIT rows: {ebit_rows}

---

### TASK
Generate a semicolon-delimited CSV table where each row represents one analytical line item of the company's general ledger.
The dataset must:
1. Follow the structure of the provided CoA.
2. Respect the exact totals above (sum per category equals the financial total).
3. Contain exactly the specified number of rows per category.
4. Include realistic business units, customers, and vendors relevant to the specific company and industry.

---

### RULES

**Columns**


In [11]:
coa_text = """1002	Finished Goods
1003	Raw Materials
1004	Work in Progress
1005	Allowance for Bad Debt
1006	Intercompany Receivables
1007	Other Receivables
1008	Trade Receivables
1009	Prepaid Expenses
1501	Land & Improvements
1502	Buildings & Improvements
1503	Leasehold Improvements
1504	Construction In Progress
1505	Machinery & Equipment
1506	Office Furniture & Equipment
2001	Deferred Taxes
2002	Goodwill
2003	Investments in Subsidiaries
2501	Accounts Payable
2502	Notes Payable
2503	Intercompany Payables
2504	Short Term Loan Obligations
2505	Salary & Other Comp
2506	Insurance
2507	Warranties
2508	CIT Payable
2509	Other Current Liabilities
3001	Long Term Loan Obligations
3002	Other Long Term Liabilities
3003	Other Retirement Benefits
3004	Pension Liability
3501	Share Capital
3502	Additional Paid In Capital
3503	Prior Year Retained Earnings
4001	Retail Gross Sales
4002	Retail Discounts
4003	Retail COS
4004	Trade Gross Sales
4005	Trade Discounts
4006	Trade COS
4007	Inter Company Gross Sales
4008	Inter Company Discounts
4009	Inter Company COS
5001	Salaries - Sales
5030	Salaries - Trainees
5031	Salaries - Other
5002	Employee Benefits
5003	Payroll Taxes
5004	Sales Commissions
5032	Bonuses
5005	Telephone
5006	Utilities
5007	Amortization of Goodwill
5008	Building Leasehold
5009	Equipment
5010	Furniture and Fixtures
5011	Property
5012	Gains/Losses On Asset Disposal - Property
5013	Repairs and Maintenance
5014	Travel Transportation
5015	Travel Lodging
5016	Entertainment
5017	Meals
5018	Other Travel Related
5019	Conferences
5020	Marketing Collateral
5021	Office Supplies
5022	Other Expenses
5023	Professional Services
5024	Rent
5025	Gains/Losses On Asset Disposal - Leasehold
5026	Gains/Losses On Asset Disposal - Plant
5027	Gains/Losses On Asset Disposal - Fixtures
5028	Gains/Losses On Asset Disposal - Goodwill
6001	Curr Xchg (Gain)/Loss
6002	Other Income
6003	Rental Income
6501	Interest Expense
6502	Interest Income
6503	Dividends Receivable
7001	Taxes
7501	Dividends Payable
9002	Units
9003	Asset Additions - Property
9004	Asset Disposal Proceeds - Property
9005	Average Unit Price
9006	Square Footage
9010	Headcount - Directors
9011	Headcount - Managers
9012	Headcount - Assistant Managers
9013	Headcount - Sales Execs
9014	Headcount - Warehouse Staff
9015	Headcount - Administrators
9016	Headcount - Trainees
9017	Headcount - Other
9018	Income Taxes Paid
9019	Interest Received
9020	Interest Paid
9021	Dividends Paid
9022	New Loans
9023	Loan Repayments
9024	Asset Additions - Leasehold
9025	Asset Disposal Proceeds - Leasehold
9026	Asset Additions - Plant
9027	Asset Disposal Proceeds - Plant
9028	Asset Additions - Fixtures
9029	Asset Disposal Proceeds - Fixtures
9030	Asset Additions - Goodwill
9031	Asset Disposal Proceeds - Goodwill"""

In [24]:
#from utils.prompts import PROMPT_LIBRARY_GL
import utils.utils as utils
from utils import prompt_utils
import numpy as np
import pandas as pd

def call_llm_prompt(prompt: str, count: int, model: str = "gpt-4.1", temp: float = 0.8) -> pd.DataFrame:
    """Executes a given prompt and parses it into a DataFrame."""
    client = prompt_utils.get_openai_client()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a financial controller generating realistic GL items."},
            {"role": "user", "content": prompt.strip()},
        ],
        temperature=temp,
    )

    df = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)

    # standard cleanup
    #df.rename(columns={"name": "item_name"}, inplace=True, errors="ignore")
    #df["unit_price"] = pd.to_numeric(df.get("unit_price", 0), errors="coerce").fillna(0.0)
    #df = utils.convert_column_to_percentage(df, "proportionality", scale=1.0)
    return df

In [10]:
import pandas as pd

df_accounts = pd.read_csv("data/inputdata/coa_general.csv", sep = ";")

In [5]:
df_accounts

,AccountKey,name,AccountType,AcSign,BudgetArea,GLLevel01,GLLevel02,GLLevel03,GLLevel04,GLLevel05,CFLevel01,CFLevel02,CFLevel03,CFLevel04,CFLevel05,CFLevel06,CFLevel07,CFLevel08
0,1001,Cash,B,1,NaN,Balance Sheet,Assets,Current Assets,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,Finished Goods,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Finished Goods,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Finished Goods,NaN
2,1003,Raw Materials,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Raw Materials,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Raw Materials,NaN
3,1004,Work in Progress,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Work in Progress,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Work in Progress,NaN
4,1005,Allowance for Bad Debt,B,1,Other Working Capital,Balance Sheet,Assets,Current Assets,Receivables,Allowance for Bad Debt,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Receivables,Allowance for Bad Debt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,9027,Asset Disposal Proceeds - Plant,S,1,PPE,Statistical Accounts,Asset Disposal Proceeds,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Disposal Proceeds,Asset Disposal Proceeds - Plant,NaN,NaN,NaN,NaN
105,9028,Asset Additions - Fixtures,S,1,PPE,Statistical Accounts,Asset Additions,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Additions,Asset Additions - Fixtures,NaN,NaN,NaN,NaN
106,9029,Asset Disposal Proceeds - Fixtures,S,1,PPE,Statistical Accounts,Asset Disposal Proceeds,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Disposal Proceeds,Asset Disposal Proceeds - Fixtures,NaN,NaN,NaN,NaN
107,9030,Asset Additions - Goodwill,S,1,PPE,Statistical Accounts,Asset Additions,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Additions,Asset Additions - Goodwill,NaN,NaN,NaN,NaN


In [9]:
tailored_coa = tailor_coa_names_llm(df_accounts, company_name=company_name)
tailored_coa.head()

KeyboardInterrupt: 

In [11]:
from generators.llm_generators import tailor_coa_names_llm

df_coa = tailor_coa_names_llm(df_accounts, company_name="Lego")

In [10]:
df_coa

,AccountKey,name,AccountType,AcSign,BudgetArea,GLLevel01,GLLevel02,GLLevel03,GLLevel04,GLLevel05,CFLevel01,CFLevel02,CFLevel03,CFLevel04,CFLevel05,CFLevel06,CFLevel07,CFLevel08
0,1001,Cash and Bank Balances,B,1,NaN,Balance Sheet,Assets,Current Assets,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,Finished LEGO Sets Inventory,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Finished Goods,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Finished Goods,NaN
2,1003,Raw Materials for Brick Production,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Raw Materials,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Raw Materials,NaN
3,1004,Work in Progress - LEGO Elements,B,1,Inventory,Balance Sheet,Assets,Current Assets,Inventory,Work in Progress,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Inventory,Work in Progress,NaN
4,1005,Allowance for Doubtful Accounts,B,1,Other Working Capital,Balance Sheet,Assets,Current Assets,Receivables,Allowance for Bad Debt,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Working Capital,Current Assets,Receivables,Allowance for Bad Debt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,9027,Plant Asset Disposal Proceeds,S,1,PPE,Statistical Accounts,Asset Disposal Proceeds,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Disposal Proceeds,Asset Disposal Proceeds - Plant,NaN,NaN,NaN,NaN
105,9028,Fixtures Asset Additions,S,1,PPE,Statistical Accounts,Asset Additions,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Additions,Asset Additions - Fixtures,NaN,NaN,NaN,NaN
106,9029,Fixtures Asset Disposal Proceeds,S,1,PPE,Statistical Accounts,Asset Disposal Proceeds,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Disposal Proceeds,Asset Disposal Proceeds - Fixtures,NaN,NaN,NaN,NaN
107,9030,Goodwill Additions,S,1,PPE,Statistical Accounts,Asset Additions,NaN,NaN,NaN,CashFlow,Cash flows From Investing Activities,Asset Additions,Asset Additions - Goodwill,NaN,NaN,NaN,NaN


In [ ]:
MASTER_PROMPT = """
You are a financial controller generating a realistic analytical spend dataset for a company.

### INPUT DATA
Company: {company_name}

Financial Totals (DKK):
- Revenue: {revenue_value}
- COGS: {cogs_value}
- Gross Profit: {gross_profit_value}
- Fixed Cost: {fixed_cost_value}
- EBIT: {ebit_value}

Chart of Accounts (CoA):
{coa_list}

Row Counts:
- Revenue rows: {revenue_rows}
- COGS rows: {cogs_rows}
- FixedCost rows: {fixedcost_rows}
- EBIT rows: {ebit_rows}

### TASK
Generate a semicolon-delimited CSV table where each row represents one analytical line item of the company's general ledger.
The dataset must:
1. Follow the structure of the provided CoA.
2. Respect the exact totals above (sum per category equals the financial total).
3. Contain exactly the specified number of rows per category.
4. Include realistic business units, customers, and vendors relevant to the specific company and industry.

---

### RULES

**Columns**
company;business_unit;account_number;account_name;item_name;customer;vendor;amount_DKK;category

**Account Mapping**
- Revenue → 4001–4009  
- COGS → 4003, 4006, 4009  
- FixedCost → 5001–5027  
- EBIT → 6001–6503  

**Category Logic**
- **Revenue**: Product or service sales.  
  - For intercompany accounts (4007–4008), `customer` is an internal business unit.  
  - For other accounts, `customer` is an external distributor or partner relevant to the company.  
- **COGS**: Production, materials, and logistics.  
  - For intercompany COGS (4009), `vendor` is an internal business unit.  
  - Otherwise, use external suppliers typical for the company's industry.  
- **FixedCost**: Overhead (salaries, marketing, rent, utilities, travel, consulting).  
- **EBIT**: Other income/expenses (interest, exchange differences, royalties, etc.).  

**Business Units**
Generate 10 realistic business units for the specific company (factories, sales regions, HQs, divisions, etc.).  
Example:
- For LEGO → LEGO Retail DK, LEGO Factory CZ, LEGO HQ Billund  
- For Arla → Arla Foods DK, Arla UK, Arla Ingredients  
- For Biocirc → Biocirc Odense, Biocirc Kalundborg, Biocirc HQ Copenhagen  

Use them consistently for internal transactions.

**Value Distribution**
Randomize amounts while ensuring that:
- Each category sums exactly to its total value.
- Variation is realistic (a few large values, many smaller).

---

### OUTPUT FORMAT
Return **only the CSV content**, starting with a header and using `;` as delimiter:
company;business_unit;account_number;account_name;item_name;customer;vendor;amount_DKK;category
Do not include explanations, markdown, or extra text—only the CSV table.

---
"""


In [22]:
prompt = MASTER_PROMPT.format(
    company_name="LEGO",
    revenue_value=102709000,
    cogs_value=-80578387,
    gross_profit_value=22130526,
    fixed_cost_value=-17916252,
    ebit_value=4214272,
    coa_list=coa_text,
    revenue_rows=100,
    cogs_rows=100,
    fixedcost_rows=50,
    ebit_rows=50
)

In [23]:
df = call_llm_prompt(prompt, count=50)

In [24]:
df

,company,business_unit,account_number,account_name,item_name,customer,vendor,amount_DKK,category
0,LEGO,LEGO Factory CZ,4007,Inter Company Gross Sales,LEGO Bricks Transfer,LEGO Retail UK,,725471,Revenue
1,LEGO,LEGO Retail DK,4004,Trade Gross Sales,LEGO DUPLO Steam Train,Bilka,,808666,Revenue
2,LEGO,LEGO Factory HU,4007,Inter Company Gross Sales,LEGO Bricks Transfer,LEGO Retail FR,,604215,Revenue
3,LEGO,LEGO Retail ES,4004,Trade Gross Sales,LEGO Speed Champions Audi,Juguettos,,132100,Revenue
4,LEGO,LEGO Retail DK,4004,Trade Gross Sales,LEGO DUPLO Town Bakery,Kids Coolshop,,43250,Revenue
5,LEGO,LEGO Factory MX,4008,Inter Company Discounts,Internal Volume Discount,LEGO Retail US,,-155098,Revenue
6,LEGO,LEGO Factory MX,4007,Inter Company Gross Sales,LEGO Bricks Transfer,LEGO Retail UK,,121000,Revenue
7,LEGO,LEGO Retail IT,4001,Retail Gross Sales,LEGO DOTS Bracelet Mega Pack,Giocheria,,70330,Revenue
8,LEGO,LEGO Retail NL,4001,Retail Gross Sales,LEGO City Fire Helicopter,Bol.com,,54021,Revenue
9,LEGO,LEGO Retail DE,4001,Retail Gross Sales,LEGO Technic Monster Jam,myToys,,38112,Revenue


In [2]:
PROMPT_TAILOR_COA = """
You are a corporate finance specialist adapting a Chart of Accounts to a specific company.

Company: {company_name}

Input Chart of Accounts:
{coa_list}

Task:
Rewrite the account descriptions and relevant GLLevel04–GLLevel05 fields to reflect this company's industry and operations.
Keep every column name and key exactly as provided.
Do not remove or add rows.

Output:
Return only a semicolon-separated CSV with identical columns.
"""


In [3]:
PROMPT_BUSINESS_UNITS = """
You are designing a realistic company structure.

Company: {company_name}

Task:
Generate 10–15 business units and departments that reflect this company’s operations.
For each, include:
BU_ID;BU_Name;BU_Type;Department;Country;Description

BU_Type examples: Factory, Retail, HQ, Licensing, Shared Service, Online.

Output:
Return only a semicolon-separated CSV with the columns above.
"""


In [4]:
PROMPT_PARTIES = """
You are populating master data for external and internal counterparties.

Company: {company_name}

Input Business Units:
{bu_list}

Task:
Generate 10–20 external customers and 10–20 external vendors, plus all internal business units as internal parties.

Columns:
Party_ID;Party_Name;Party_Type;Country;Description

Where Party_Type ∈ {INTERNAL_BU, CUSTOMER, VENDOR}.
Use internal BUs from the list above for Party_Type=INTERNAL_BU.

Output:
Return only a semicolon-separated CSV.
"""


In [12]:
PROMPT_LINES = """
You are generating detailed GL line items for synthetic analytical spend data.

Company: {company_name}

Financial total (DKK): {financial_total}
Category: {category_name}
Number of rows: {row_count}

Input context:
Chart of Accounts (relevant subset):
{coa_subset}

Business Units (IDs & Names):
{bu_list}

Parties (Customers/Vendors):
{party_list}

Task:
Generate {row_count} detailed GL rows for the {category_name} category, that sum exactly to {financial_total} DKK.

Rules:
- Use only account_numbers from the provided CoA subset.
- Reference valid BU_ID and Party_ID.
- Each row must include:
  document_number;posting_date;company;BU_ID;Party_ID;account_number;account_name;item_name;amount_DKK;category
- posting_date: random across 2024.
- document_number: DOC000001 … sequential.
- Intercompany lines must use internal BUs for Party_ID.

Output:
Return only a semicolon-separated CSV with the columns above.
"""


In [29]:
#### llm_chain.py

import pandas as pd
from utils.prompts import call_llm_prompt
#from llm_generation.prompts import (
#    PROMPT_TAILOR_COA, PROMPT_BUSINESS_UNITS,
#    PROMPT_PARTIES, PROMPT_LINES
#)

def generate_demo_world(company, coa_df, finances, row_cfg):
    ctx = {}

    # Step 1 – Tailor CoA
    prompt_coa = PROMPT_TAILOR_COA.format(company_name=company, coa_list=coa_df.to_csv(index=False, sep=";"))
    ctx["coa"] = call_llm_prompt(prompt_coa, count=len(coa_df))

    # Step 2 – Business Units
    prompt_bu = PROMPT_BUSINESS_UNITS.format(company_name=company)
    ctx["bus"] = call_llm_prompt(prompt_bu, count=15)

    # Step 2.5 – Parties
    prompt_parties = PROMPT_PARTIES.format(company_name=company, bu_list=ctx["bus"].to_csv(index=False, sep=";"))
    ctx["parties"] = call_llm_prompt(prompt_parties, count=40)

    # Step 3–4 – Financial Categories
    categories = ["Revenue", "COGS", "FixedCost", "EBIT"]
    for cat in categories:
        prompt_lines = PROMPT_LINES.format(
            company_name=company,
            category_name=cat,
            financial_total=finances[cat],
            row_count=row_cfg[cat],
            coa_subset=ctx["coa"].query("GLLevel03.str.contains(@cat, case=False, na=False)").to_csv(index=False, sep=";"),
            bu_list=ctx["bus"].to_csv(index=False, sep=";"),
            party_list=ctx["parties"].to_csv(index=False, sep=";")
        )
        ctx[cat] = call_llm_prompt(prompt_lines, count=row_cfg[cat])

    return ctx

NameError: name 'over_request_count' is not defined

In [13]:
PROMPT_BUSINESS_UNITS = """
You are creating a realistic internal org structure for a company.

Company: {company_name}

Task:
Generate 10-15 business units and departments that reflect how this company would actually operate (production sites, regional sales orgs, HQ functions, logistics hubs, shared services, etc.).

Return ONLY a semicolon-separated CSV with the following columns in this exact order:
BU_ID;BU_Name;BU_Type;Department;Country;Description

Definitions:
- BU_ID: stable ID like BU001, BU002, ...
- BU_Name: human label, e.g. "LEGO Retail DK", "LEGO Factory CZ", "LEGO HQ Billund"
- BU_Type: one of [Factory, Retail, HQ, Licensing, Shared Service, Online, Distribution]
- Department: e.g. "Sales", "Manufacturing", "Finance", "Marketing", "Logistics", "IT"
- Country: realistic country/region for that BU
- Description: short purpose of this unit

Rules:
- Make sure there is at least one HQ / corporate finance unit.
- Make sure there are both commercial (sales/retail) and production/supply-side units.
- IDs must be unique.
- Do not include any extra commentary, only the CSV.
"""


In [14]:
PROMPT_PARTIES = """
You are creating master data for all counterparties involved in billing and cost of goods.

Company: {company_name}

Here is the list of internal business units for this company:
{business_units_csv}

Task:
1. For each internal BU above, create a row where that BU is a party.
2. Create 10-20 external customers (distributors, retailers, key accounts, web shop channels).
3. Create 10-20 external vendors (material suppliers, logistics providers, energy providers, maintenance contractors).

Return ONLY a semicolon-separated CSV with the following columns in this exact order:
Party_ID;Party_Name;Party_Type;Country;Description

Where:
- Party_ID:
  - INTERNAL_BU should start with "INT" (e.g. INT001)
  - CUSTOMER should start with "CUST" (e.g. CUST001)
  - VENDOR should start with "VEND" (e.g. VEND001)
- Party_Type is one of [INTERNAL_BU, CUSTOMER, VENDOR]
- Country and Description should be realistic
- Party_Name MUST stay stable and professional (e.g. "Amazon EU Retail", "Maersk Logistics DK", "LEGO Retail DK")

Rules:
- For INTERNAL_BU parties, Party_Name should exactly match the BU_Name from the input business units.
- INTERNAL_BU rows must cover ALL input BU_IDs.
- Do not include any extra commentary, only the CSV.
"""


In [15]:
PROMPT_LINES = """
You are generating detailed GL/transaction lines for synthetic financial data.

Company: {company_name}
Category to generate: {category_name}        # e.g. Revenue, COGS, FixedCost, EBIT
Target total for this category (DKK): {financial_total}
Number of rows to generate: {row_count}

ACCOUNTS (only use these AccountKeys for this category):
AccountKey;name
{accounts_subset_csv}

BUSINESS UNITS (use these BU_IDs only):
BU_ID;BU_Name;BU_Type;Department;Country;Description
{business_units_csv}

PARTIES (customers, vendors, internal units):
Party_ID;Party_Name;Party_Type;Country;Description
{parties_csv}

Your task:
Generate {row_count} GL line items for the given category {category_name}.
The sum of 'amount_DKK' over ALL {row_count} rows MUST equal {financial_total} exactly.

Output a semicolon-separated CSV with columns in this exact order:
document_number;posting_date;company;BU_ID;Party_ID;AccountKey;AccountName;item_name;amount_DKK;category

Column definitions:
- document_number: sequential, zero-padded: DOC000001, DOC000002, ...
- posting_date: realistic YYYY-MM-DD dates across the year 2024
- company: literal company name
- BU_ID: must match one of the BU_ID values from the business units table
- Party_ID:
  - For Revenue:
    - If AccountKey is an intercompany revenue account (e.g. "Inter Company Gross Sales"), Party_ID must be an INTERNAL_BU.
    - Otherwise use CUSTOMER.
  - For COGS:
    - If AccountKey is an intercompany COGS account, Party_ID must be an INTERNAL_BU.
    - Otherwise use VENDOR.
  - For FixedCost and EBIT:
    - Party_ID can be blank unless it clearly represents a counterparty (like external legal services).
- AccountKey: must be copied from the provided accounts list
- AccountName: must be copied from the 'name' column for that AccountKey
- item_name: human-readable description of what the line is (examples: "ABS plastic pellets", "Retail set sales - City/Police Station", "Factory utilities Denmark", "Royalty income from licensing")
- amount_DKK: numeric, can be positive or negative depending on category
- category: must match {category_name} exactly

Rules:
- DO NOT invent new AccountKeys. Only use the provided ones.
- DO NOT invent new BU_IDs or Party_IDs. Only use the provided ones.
- Make item_name specific and believable for this industry.
- The sum of all amount_DKK in this output must equal {financial_total} exactly.
- You must return EXACTLY {row_count} rows.
- Do not include any commentary, markdown, headers before the header row etc.

Return ONLY the CSV.
"""


In [ ]:
import pandas as pd
#from utils import prompt_utils
#from utils.utils import convert_column_to_percentage
#from llm_generation.prompts import PROMPT_BUSINESS_UNITS, PROMPT_PARTIES, PROMPT_LINES
#from utils.prompts import call_llm_prompt  # your existing function

# helper to convert df to ; separated text
def df_to_semicolon_csv(df: pd.DataFrame) -> str:
    return df.to_csv(index=False, sep=";")

def generate_business_units(company_name: str) -> pd.DataFrame:
    prompt = PROMPT_BUSINESS_UNITS.format(company_name=company_name)
    return call_llm_prompt(prompt, prompt, count=20)

def generate_parties(company_name: str, bu_df: pd.DataFrame) -> pd.DataFrame:
    prompt = PROMPT_PARTIES.format(
        company_name=company_name,
        business_units_csv=df_to_semicolon_csv(bu_df)
    )
    return call_llm_prompt(prompt, count=60)

def generate_category_lines(
    company_name: str,
    category_name: str,
    financial_total: float,
    row_count: int,
    accounts_subset_df: pd.DataFrame,
    bu_df: pd.DataFrame,
    parties_df: pd.DataFrame
) -> pd.DataFrame:
    prompt = PROMPT_LINES.format(
        company_name=company_name,
        category_name=category_name,
        financial_total=financial_total,
        row_count=row_count,
        accounts_subset_csv=df_to_semicolon_csv(accounts_subset_df[["AccountKey","name"]]),
        business_units_csv=df_to_semicolon_csv(bu_df),
        parties_csv=df_to_semicolon_csv(parties_df)
    )
    return call_llm_prompt(prompt, count=row_count)


In [38]:
def generate_vendors_llm(company_name: str, count: int = 100, model: str = "gpt-4.1", temp: float = 0.8):
    client = prompt_utils.get_openai_client()
    over_request_count = int(np.floor(int(count) * 1.4))
    header = "name;vendor_type;proportionality"
    constraints = prompt_utils.get_standard_constraints(header, over_request_count)
    ctxb = prompt_utils._ctx_block(company_name)

    prompt = f"""
    You are a B2B procurement and supply chain expert. Generate {over_request_count} realistic vendors for {company_name}.
    Fields:
    - name
    - vendor_type: Raw Materials, Equipment, IT Services, Logistics, Facilities, Office Supplies, Contract Labor, Consulting
    - proportionality: the proportionality of this vendor.
    Bias critical categories and concentration based on the context.

    {constraints}
    """.strip()

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful data assistant and B2B vendor segmentation expert."},
                  {"role": "user", "content": prompt}],
        temperature=temp,
    )
    df_vendors = prompt_utils.parse_and_truncate_csv(response.choices[0].message.content, count)
    df_vendors.insert(0, "vendor_id", range(20, len(df_vendors) + 20))
    df_vendors = utils.convert_column_to_percentage(df_vendors, "proportionality", scale=1.0)
    return df_vendors

In [25]:
def call_llm_prompt(
    prompt: str,
    model: str = "gpt-4.1",
    temp: float = 0.8,
) -> str:
    """
    Executes a given prompt and returns the raw model text output.
    No parsing. No truncation.
    """
    client = prompt_utils.get_openai_client()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a financial controller generating realistic GL items."
            },
            {
                "role": "user",
                "content": prompt.strip()
            },
        ],
        temperature=temp,
    )
    
    return response.choices[0].message.content


In [17]:
import numpy as np
import pandas as pd

def call_llm_overrequest(
    prompt_template: str,
    prompt_kwargs: dict,
    count: int,
    model: str = "gpt-5",
    temp: float = 1,
    over_factor: float = 1.4
) -> pd.DataFrame:
    """
    Renders the prompt, over-requests rows from the model,
    parses them, and returns a df with exactly `count` rows
    (or raises if even that is impossible).
    """
    # how many rows we *ask* the LLM for
    over_request_count = int(np.floor(int(count) * over_factor))

    # inject both requested count and over_request_count into the prompt
    full_prompt = prompt_template.format(
        **prompt_kwargs,
        count=count,
        over_request_count=over_request_count,
    )

    # 1) get raw text from LLM
    raw_text = call_llm_prompt(
        prompt=full_prompt,
        model=model,
        temp=temp,
    )

    # 2) parse whatever it gave us into a df
    # IMPORTANT: we pass over_request_count here because that's the "max rows" we
    # tried to ask for. parse_and_truncate_csv should NOT raise hard if it's short.
    df_full = prompt_utils.parse_and_truncate_csv(
        raw_text,
        over_request_count
    )

    # 3) sanity check: did we at least get `count` usable rows?
    if len(df_full) < count:
        raise ValueError(
            f"LLM underfilled: needed {count} rows but only got {len(df_full)} usable rows "
            f"after requesting {over_request_count}."
        )

    # 4) now truncate to exactly the number the caller wanted
    df = df_full.head(count).reset_index(drop=True)

    return df

In [18]:
PROMPT_BUSINESS_UNITS = """
You are creating a realistic internal org structure for a company.

Company: {company_name}

Task:
Generate {over_request_count} business units / departments that reflect how this company operates
(production sites, regional sales orgs, HQ functions, logistics hubs, shared services, etc.).

Return ONLY a semicolon-separated CSV with the following columns in this exact order:
BU_ID;BU_Name;BU_Type;Department;Country,Company_ID

Rules:
- BU_ID: stable ID like BU001, BU002, ...
- BU_Type: one of [Factory, Retail, HQ, Licensing, Shared Service, Online, Distribution]
- At least one HQ / Corporate / Finance function must exist.
- Include both commercial (sales/retail) and production/supply-side units.
- IDs must be unique.
- For each Unique country there should be a unique companyID. Start with 1000.
- Do not add commentary or markdown, only the CSV.
"""

In [19]:
def generate_business_units(company_name: str,
                            count: int = 15,
                            model: str = "gpt-5",
                            temp: float = 1) -> pd.DataFrame:
    prompt_kwargs = {
        "company_name": company_name,
    }
    df_bu = call_llm_overrequest(
        prompt_template=PROMPT_BUSINESS_UNITS,
        prompt_kwargs=prompt_kwargs,
        count=count,
        model=model,
        temp=temp,
        over_factor=1.4
    )
    return df_bu


In [20]:
PROMPT_PARTIES = """
You are creating master data for all counterparties in this company.

Company: {company_name}

Internal business units (BU master data):
{business_units_csv}

Task:
1. For each internal BU, create a row where that BU is treated as a party.
2. Also create external customers (distributors, retailers, channels).
3. Also create external vendors (materials suppliers, logistics, energy, maintenance, IT services).

Generate {over_request_count} rows TOTAL across all types.

Return ONLY a semicolon-separated CSV with columns in this exact order:
Party_ID;Party_Name;Party_Type;Country

Where:
- Party_ID:
  - INTERNAL_BU => "INT###"
  - CUSTOMER    => "CUST###"
  - VENDOR      => "VEND###"
- Party_Type is exactly one of [INTERNAL_BU, CUSTOMER, VENDOR]
- INTERNAL_BU rows must include ALL internal business units given above.
- Party_Name for INTERNAL_BU must match BU_Name exactly.
- No commentary, no markdown, only CSV.
"""


In [21]:
def generate_parties(
    company_name: str,
    bu_df: pd.DataFrame,
    count: int = 40,
    model: str = "gpt-5",
    temp: float = 1
) -> pd.DataFrame:
    prompt_kwargs = {
        "company_name": company_name,
        "business_units_csv": bu_df.to_csv(index=False, sep=";"),
    }

    df_parties = call_llm_overrequest(
        prompt_template=PROMPT_PARTIES,
        prompt_kwargs=prompt_kwargs,
        count=count,
        model=model,
        temp=temp,
        over_factor=1.4,
    )
    return df_parties

In [ ]:
PROMPT_LINES = """
You are generating detailed GL/transaction lines for synthetic financial data.

Company: {company_name}
Category to generate: {category_name}        
Target total for this category (DKK): {financial_total}
We ultimately need {count} rows, but you must generate {over_request_count} rows so we can select the best subset.

ACCOUNTS (only use these AccountKeys for this category):
AccountKey;name
{accounts_subset_csv}

BUSINESS UNITS (valid BU_IDs only):
BU_ID;BU_Name;BU_Type;Department;Country;Description
{business_units_csv}

PARTIES (valid Party_IDs only):
Party_ID;Party_Name;Party_Type;Country;Description
{parties_csv}

Task:
Generate {over_request_count} GL line items for the given category {category_name}.
The sum of 'amount_DKK' across ALL {over_request_count} rows MUST equal {financial_total} exactly.

Output a semicolon-separated CSV with columns in this exact order:
document_number;posting_date;company;BU_ID;Party_ID;AccountKey;AccountName;item_name;amount_DKK;category

Column definitions:
- document_number: sequential, zero-padded: 000001, 000002, ...
- company: literal company name
- BU_ID: must match one of the BU_ID values from the business units table
- Party_ID:
  - Revenue:
    - If AccountKey relates to intercompany Revenue, Party_ID must be an INTERNAL_BU.
    - Otherwise must be CUSTOMER.
  - COGS:
    - If AccountKey relates to intercompany COGS, Party_ID must be an INTERNAL_BU.
    - Otherwise must be VENDOR.
  - FixedCost and EBIT:
    - Party_ID can be blank unless there's a clear external counterparty (e.g. law firm)
- AccountKey: must come from ACCOUNTS section
- AccountName: must be copied from 'name' in ACCOUNTS
- item_name: specific and believable for this industry (materials, sets sold, marketing campaign, royalty income, etc.)
- amount_DKK: numeric, signs consistent with category
- category: must match {category_name} exactly

Rules:
- DO NOT invent new AccountKeys, BU_IDs, or Party_IDs.
- You must return EXACTLY {over_request_count} rows.
- The total sum of amount_DKK across ALL rows you output must equal {financial_total} exactly.
- No commentary, markdown fences, or explanations. Only CSV.
"""


In [37]:
import pandas as pd

def generate_category_lines(
    company_name: str,
    category_name: str,
    financial_total: float,
    row_count: int,
    accounts_subset_df: pd.DataFrame,
    bu_df: pd.DataFrame,
    parties_df: pd.DataFrame,
    model: str = "gpt-5",
    temp: float = 1,
    over_factor: float = 1.4,
) -> pd.DataFrame:
    """
    Generate granular line items for a given P&L category (e.g. 'Revenue').

    - We call the LLM with overrequest, *but we will not die* if it returns
      fewer than row_count rows. We'll pad ourselves.
    - We then force the final df to have exactly `row_count` rows.
    - Finally we reconcile amounts to sum to `financial_total`.
    """

    desired_rows = int(row_count)

    # --- 1. Prep prompt kwargs ---
    accounts_subset_limited = accounts_subset_df.head(100).copy()

    prompt_kwargs = {
        "company_name": company_name,
        "category_name": category_name,
        "financial_total": financial_total,
        "accounts_subset_csv": accounts_subset_limited[["AccountKey", "name"]]
            .to_csv(index=False, sep=";"),
        "business_units_csv": bu_df.to_csv(index=False, sep=";"),
        "parties_csv": parties_df.to_csv(index=False, sep=";"),
    }

    # --- 2. Call LLM (it will over-request internally), but DO NOT TRUST IT ---
    try:
        df_lines = call_llm_overrequest(
            prompt_template=PROMPT_LINES,
            prompt_kwargs=prompt_kwargs,
            count=desired_rows,
            model=model,
            temp=temp,
            over_factor=over_factor,
        )
    except ValueError as e:
        # salvage mode:
        # call_llm_overrequest is yelling "Only received 139 rows, expected 140"
        # but it probably *has* the partial df somewhere before raising.
        # If it doesn't expose that, we just retry a simpler call without over_factor
        # and then continue. We MUST return *something*.
        # We'll assume we can still query the model once without strict check.
        # If you already have a "bare" LLM caller, plug it in here.
        df_lines = basic_llm_call_no_overrequest(
            prompt_template=PROMPT_LINES,
            prompt_kwargs=prompt_kwargs,
            count=desired_rows,
            model=model,
            temp=temp,
        )

        # If that still fails for some reason and gives nothing, create empty shell:
        if df_lines is None or len(df_lines) == 0:
            df_lines = pd.DataFrame(
                [{
                    "description": f"{category_name} line {i+1}",
                    "AccountKey": None,
                    "BusinessUnit": None,
                    "Counterparty": None,
                    "amount_DKK": 0.0,
                } for i in range(desired_rows)]
            )

    # --- 3. Normalize/clean columns (especially amount) ---
    def _clean(col):
        return (
            str(col)
            .strip()
            .lower()
            .replace(" ", "_")
            .replace("-", "_")
        )

    colmap_clean = { _clean(c): c for c in df_lines.columns }

    amount_candidates_clean = [
        "amount_dkk",
        "amount",
        "dkk_amount",
        "value_dkk",
        "value",
        "line_amount",
        "line_value",
    ]

    amount_col_original = None
    for cand in amount_candidates_clean:
        if cand in colmap_clean:
            amount_col_original = colmap_clean[cand]
            break

    if amount_col_original is None:
        # heuristic fallback: pick the numeric column with biggest absolute sum
        numeric_cols = []
        for c in df_lines.columns:
            s_num = pd.to_numeric(df_lines[c], errors="coerce")
            if s_num.notna().any():
                numeric_cols.append((c, s_num.abs().sum()))
        if numeric_cols:
            numeric_cols.sort(key=lambda x: x[1], reverse=True)
            amount_col_original = numeric_cols[0][0]

    if amount_col_original is not None:
        df_lines["amount_DKK"] = pd.to_numeric(
            df_lines[amount_col_original], errors="coerce"
        ).fillna(0.0)
    else:
        df_lines["amount_DKK"] = 0.0

    # --- 4. Force EXACTLY desired_rows ---
    if len(df_lines) > desired_rows:
        # too many -> crop
        df_lines = df_lines.iloc[:desired_rows].copy()

    elif len(df_lines) < desired_rows:
        if len(df_lines) == 0:
            df_lines = pd.DataFrame(
                [{
                    "description": f"{category_name} line {i+1}",
                    "AccountKey": None,
                    "BusinessUnit": None,
                    "Counterparty": None,
                    "Amount": 0.0,
                } for i in range(desired_rows)]
            )
        else:
            last_row = df_lines.iloc[[-1]].copy()
            last_row["Amount"] = 0.0
            pads_needed = desired_rows - len(df_lines)
            pads = pd.concat([last_row.copy() for _ in range(pads_needed)],
                             ignore_index=True)
            df_lines = pd.concat([df_lines, pads], ignore_index=True)

    # safety assert: if this trips something is *really* off
    if len(df_lines) != desired_rows:
        raise RuntimeError(
            f"After salvage/pad we still don't have {desired_rows} rows "
            f"(got {len(df_lines)})"
        )

    # --- 5. Reconcile numeric amounts to financial_total ---
    df_lines["Amount"] = pd.to_numeric(
        df_lines["Amount"], errors="coerce"
    ).fillna(0.0)

    current_sum = float(df_lines["Amount"].sum())

    if current_sum != 0:
        scale = float(financial_total) / current_sum
        df_lines["Amount"] = df_lines["Amount"] * scale
    else:
        if financial_total != 0 and desired_rows > 0:
            even_share = float(financial_total) / float(desired_rows)
            df_lines["Amount"] = even_share

    df_lines["Amount"] = df_lines["Amount"].round(2)

    final_sum_after_round = float(df_lines["Amount"].sum())
    diff = float(financial_total) - final_sum_after_round
    if desired_rows > 0:
        df_lines.loc[desired_rows - 1, "Amount"] += diff

    df_lines = df_lines.reset_index(drop=True)

    return df_lines


In [28]:
# === Step 1: load your general df_accounts ===
#df_accounts = pd.read_csv("path/to/df_accounts.csv", sep=";")
df_accounts = df_coa.copy()

# === Step 2: Generate Business Units ===
company_name = "LEGO"
bu_df = generate_business_units(company_name)
print(bu_df.head())

   BU_ID                              BU_Name         BU_Type  \
0  BU001              LEGO Group Headquarters              HQ   
1  BU002  Global Finance & Controlling Center  Shared Service   
2  BU003        Global IT & Digital Platforms  Shared Service   
3  BU004   Product Design & Innovation Studio              HQ   
4  BU005            Billund Molding & Tooling         Factory   

                Department  Country  Company_ID  
0       Corporate Strategy  Denmark        1000  
1  Finance Shared Services  Denmark        1000  
2   Information Technology  Denmark        1000  
3             Design & R&D  Denmark        1000  
4        Injection Molding  Denmark        1000  


In [39]:
bu_df

,BU_ID,BU_Name,BU_Type,Department,Country,Company_ID
0,BU001,LEGO Group Headquarters,HQ,Corporate Strategy,Denmark,1000
1,BU002,Global Finance & Controlling Center,Shared Service,Finance Shared Services,Denmark,1000
2,BU003,Global IT & Digital Platforms,Shared Service,Information Technology,Denmark,1000
3,BU004,Product Design & Innovation Studio,HQ,Design & R&D,Denmark,1000
4,BU005,Billund Molding & Tooling,Factory,Injection Molding,Denmark,1000
5,BU006,Kladno Packaging Plant,Factory,Packaging & Decoration,Czech Republic,1001
6,BU007,Central Europe Distribution Hub - Prague,Distribution,Regional Distribution,Czech Republic,1001
7,BU008,Nyíregyháza Manufacturing,Factory,Assembly & Packaging,Hungary,1002
8,BU009,Monterrey Manufacturing Campus,Factory,Injection Molding,Mexico,1003
9,BU010,Monterrey Americas Distribution Center,Distribution,Regional Distribution,Mexico,1003


In [29]:
# === Step 3: Generate Master Parties ===
parties_df = generate_parties(company_name, bu_df)
print(parties_df.head())

  Party_ID                           Party_Name   Party_Type  Country
0   INT001              LEGO Group Headquarters  INTERNAL_BU  Denmark
1   INT002  Global Finance & Controlling Center  INTERNAL_BU  Denmark
2   INT003        Global IT & Digital Platforms  INTERNAL_BU  Denmark
3   INT004   Product Design & Innovation Studio  INTERNAL_BU  Denmark
4   INT005            Billund Molding & Tooling  INTERNAL_BU  Denmark


In [38]:
# === Step 4: Prepare your financial totals ===
finances = {
    "Revenue": 102_709_000,
    "COGS": -80_578_387,
    "FixedCost": -17_916_252,
    "EBIT": 4_214_272
}
rows_cfg = {"Revenue": 100, "COGS": 100, "FixedCost": 50, "EBIT": 50}

# === Step 5: Run one category at a time ===
revenue_accounts = df_accounts[df_accounts["AccountKey"].between(4001, 4009)]
revenue_df = generate_category_lines(
    company_name,
    "Revenue",
    finances["Revenue"],
    rows_cfg["Revenue"],
    revenue_accounts,
    bu_df,
    parties_df
)

Only received 139 rows, expected 140


TypeError: object of type 'int' has no len()

In [ ]:
revenue_df

,document_number,posting_date,company,BU_ID,Party_ID,AccountKey,AccountName,item_name,amount_DKK,category
0,DOC000001,2024-01-03,LEGO,BU014,CUST003,4001,LEGO Retail Gross Sales,Retail POS sales - North America - Amazon mark...,948703.38,Revenue
1,DOC000002,2024-01-04,LEGO,BU015,CUST001,4001,LEGO Retail Gross Sales,Retail POS sales - DACH & Nordics - Walmart ma...,948703.38,Revenue
2,DOC000003,2024-01-05,LEGO,BU014,CUST002,4001,LEGO Retail Gross Sales,Retail POS sales - North America - Target onli...,948703.38,Revenue
3,DOC000004,2024-01-06,LEGO,BU015,CUST005,4001,LEGO Retail Gross Sales,Retail POS sales - DACH & Nordics - Smyths Toy...,948703.38,Revenue
4,DOC000005,2024-01-07,LEGO,BU014,CUST012,4001,LEGO Retail Gross Sales,Retail POS sales - North America - MediaMarkt ...,948703.38,Revenue
...,...,...,...,...,...,...,...,...,...,...
95,DOC000113,2024-05-16,LEGO,BU013,CUST005,4004,Trade Channel Gross Sales,Order - Smyths - Half-term,1304467.15,Revenue
96,DOC000114,2024-05-17,LEGO,BU013,CUST006,4004,Trade Channel Gross Sales,Window - The Entertainer - Summer line-up,1304467.15,Revenue
97,DOC000115,2024-05-18,LEGO,BU013,CUST008,4004,Trade Channel Gross Sales,Argos - Catalogue refresh,1304467.15,Revenue
98,DOC000116,2024-05-19,LEGO,BU013,CUST009,4004,Trade Channel Gross Sales,Tesco - Toy aisle expansion,1304467.15,Revenue


In [31]:
PROMPT_LINES = """
You are a financial data generator simulating {company_name}'s general ledger for the category "{category_name}".

The total amount across all rows must approximately equal {financial_total} DKK.

Generate {over_request_count} detailed line items that could realistically appear in the general ledger for this category.

Use the following context tables to inspire your results:
- Chart of accounts (AccountKey;name):
{accounts_subset_csv}

- Business units (semicolon-separated CSV):
{business_units_csv}

- Relevant counterparties (semicolon-separated CSV):
{parties_csv}

Return a semicolon-separated table (no markdown, no commentary) with these columns:
name;AccountKey;AccountName;BusinessUnit;PartyName;Quantity;UnitPrice;Amount;Description

Rules:
- "name" should be a realistic product, item, or service name (e.g., "Lego Duplo Set", "Consulting Fee", "Electricity Invoice").
- "AccountKey" must be one from the accounts_subset_csv.
- "Amount" should align with UnitPrice * Quantity, and the total should be close to {financial_total}.
- Ensure diversity in items, parties, and business units.
- Do NOT include headers or explanations outside the CSV table.
"""


In [101]:
df_lines = generate_category_lines(
    company_name="LEGO Group",
    category_name="Revenue",
    financial_total=1_500_000,
    row_count=40,
    accounts_subset_df=df_accounts,
    bu_df=bu_df,
    parties_df=parties_df,
)

Only received 55 rows, expected 56


TypeError: object of type 'int' has no len()